In [1]:
import numpy as np
import nltk
import re
from nltk.tag.perceptron import PerceptronTagger
from collections import Counter
from collections import defaultdict

In [2]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",

                           "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",

                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",

                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",

                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",

                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",

                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",

                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",

                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", 
                       
                            "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",

                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",

                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",

                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",

                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",

                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",

                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",

                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",

                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", 
                    
                            "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",

                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",

                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",

                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",

                           "you're": "you are", "you've": "you have"}

In [3]:
tagger = PerceptronTagger()
tagset = 'universal'

In [4]:
tokens = nltk.word_tokenize('the mat sat on the cat')
tags = nltk.tag._pos_tag(tokens, tagset, tagger, lang='eng')
tags

[('the', 'DET'),
 ('mat', 'NOUN'),
 ('sat', 'VERB'),
 ('on', 'ADP'),
 ('the', 'DET'),
 ('cat', 'NOUN')]

In [5]:
def preprocess(text):
    text = text.lower()
    text = re.sub(r"(\w) '(\w)", r"\1'\2", text)  # remove the space before the apostrophe
    text = re.sub(r" n't", "n't", text)
    # print(text)
    text = re.sub(r'\(.*\)', '', text)            # remove word in parentheses
    
    
    words = text.split(' ')
    result = []
    for i in range(len(words)):
        if words[i] in contraction_mapping: 
            result += contraction_mapping[words[i]].split(' ')
            continue
        result.append(words[i])
            
    text = ' '.join(result)
    text = text.replace("'s",'')
    text = re.sub(r'[^a-zA-Z0-9 ]','',text)      # remove punctuations
    return text

In [6]:
preprocess("you take paul 's turn because he ca n't do that but I 'll do that since it 's a nice day.")

'you take paul turn because he cannot do that but i will do that since it is a nice day'

In [11]:
sentences = []
with open("../../../data/all_sents_shuffled.raw", 'r') as file: 
    for line in file: 
        line = line.strip()
        sentences.append(preprocess(line)) 

In [14]:
sentences[100:105], len(sentences)

(['you stepped on my calf and that hurt',
  'is not he cute',
  'they are playing in your room',
  'clucko clucko clucko do you live up in the mountains',
  'we can play a couple games'],
 223675)

In [13]:
tokenized_dataset = []
for sent in sentences: 
    tokens = nltk.word_tokenize(sent)
    tags = nltk.tag._pos_tag(tokens, tagset, tagger, lang='eng')
    tokenized_dataset.append(tags)

In [10]:
tokenized_dataset[:5]

[[('you', 'PRON'),
  ('take', 'VERB'),
  ('paul', 'ADJ'),
  ('turn', 'NOUN'),
  ('because', 'ADP'),
  ('he', 'PRON'),
  ('can', 'VERB'),
  ('not', 'ADV'),
  ('do', 'VERB'),
  ('that', 'ADP')],
 [('you', 'PRON')],
 [('my', 'PRON'),
  ('this', 'DET'),
  ('is', 'VERB'),
  ('a', 'DET'),
  ('big', 'ADJ'),
  ('bed', 'NOUN')],
 [('yes', 'NOUN')],
 [('huh', 'NOUN')]]

In [116]:
def count_length(dataset, length):
    count = 0
    for sent in dataset: 
        if len(sent) == length: 
            count += 1
    return count

In [117]:
count_length(tokenized_dataset, 1), count_length(tokenized_dataset, 2), count_length(tokenized_dataset, 3), len(tokenized_dataset)

(46602, 22274, 30116, 223675)

In [71]:
def filter_length(dataset, filter_len=7):
    return [sent for sent in dataset if len(sent) >= filter_len]

In [72]:
filtered_dataset = filter_length(tokenized_dataset)
len(filtered_dataset)

47246

In [59]:
def replace_low_frequency_words(sentences_with_pos_tags, filter_count=1):
    # Count the frequencies of each word
    word_counts = Counter(word for sentence in sentences_with_pos_tags for word, _ in sentence)

    # Replace words with count less than filter_count to 'UNK' and their tags to 'UNK_TAG'
    processed_sentences = []
    for sentence in sentences_with_pos_tags:
        new_sentence = []
        for word, pos in sentence:
            if pos in ['X', '.']:
                continue
            if word_counts[word] < filter_count:
                new_word = 'UNK'                # Only set the word to UNK
                new_pos = pos
            else:
                new_word = word
                new_pos = pos
            new_sentence.append((new_word, new_pos))
        processed_sentences.append(new_sentence)

    return processed_sentences

In [73]:
filtered_sentences = replace_low_frequency_words(filtered_dataset, filter_count=10)
filtered_sentences[:3]

[[('you', 'PRON'),
  ('take', 'VERB'),
  ('paul', 'ADJ'),
  ('turn', 'NOUN'),
  ('because', 'ADP'),
  ('he', 'PRON'),
  ('can', 'VERB'),
  ('not', 'ADV'),
  ('do', 'VERB'),
  ('that', 'ADP')],
 [('let', 'VERB'),
  ('us', 'PRON'),
  ('get', 'VERB'),
  ('out', 'ADP'),
  ('of', 'ADP'),
  ('the', 'DET'),
  ('bathroom', 'NOUN')],
 [('when', 'ADV'),
  ('nana', 'NOUN'),
  ('comes', 'VERB'),
  ('down', 'ADV'),
  ('to', 'PRT'),
  ('the', 'DET'),
  ('UNK', 'NOUN'),
  ('office', 'NOUN'),
  ('and', 'CONJ'),
  ('get', 'VERB'),
  ('it', 'PRON')]]

In [61]:
def create_vocab_index(sentences_with_pos_tags):
    # Function to create a dictionary mapping each unique word/POS to an integer index
    # with specified start index
    def build_index(items, start_index=0):
        item_to_index = defaultdict(lambda: len(item_to_index) + start_index)
        for item in items:
            item_to_index[item]
        return dict(item_to_index)

    all_words = [word for sentence in sentences_with_pos_tags for word,_ in sentence]
    all_pos_tags = [pos for sentence in sentences_with_pos_tags for _, pos in sentence]

    return build_index(all_words, start_index=0), build_index(all_pos_tags, start_index=1)

In [74]:
word_to_index, pos_to_index = create_vocab_index(filtered_sentences)
print(len(word_to_index))
list(word_to_index.items())[10:15], list(pos_to_index.items())

1830


([('let', 10), ('us', 11), ('get', 12), ('out', 13), ('of', 14)],
 [('PRON', 1),
  ('VERB', 2),
  ('ADJ', 3),
  ('NOUN', 4),
  ('ADP', 5),
  ('ADV', 6),
  ('DET', 7),
  ('PRT', 8),
  ('CONJ', 9),
  ('NUM', 10)])

In [19]:
def count_tags(tag, sentences_with_pos_tags):
    count = 0
    for sentence in sentences_with_pos_tags: 
        for _, pos in sentence:
            if pos == tag: 
                print(sentence)
                count += 1
                
    return count

In [20]:
count_tags(".", filtered_sentences)

0

In [63]:
def convert_to_indexes(filtered_sentences_tags, word_to_index, pos_to_index):
    hidden_states_universal = []
    observations = []

    for sentence in filtered_sentences_tags:
        # if len(sentence) <= 5: 
        #     continue
        sentence_pos_indexes = [pos_to_index[pos] for _, pos in sentence]
        sentence_word_indexes = [word_to_index[word] for word, _ in sentence]

        hidden_states_universal.append(sentence_pos_indexes)
        observations.append(sentence_word_indexes)

    return hidden_states_universal, observations

In [75]:
hidden_states_universal, observations = convert_to_indexes(
    filtered_sentences, word_to_index, pos_to_index)

In [65]:
for index in range(len(observations[:5])):
    print('[' + ', '.join(map(str, hidden_states_universal[:5][index])) + ']')
    print('[' + ', '.join(map(str, observations[:5][index])) + ']')
    print('-----------------------------')

[1, 2, 3, 4, 5, 1, 2, 6, 2, 5]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
-----------------------------
[1, 7, 2, 7, 3, 4]
[10, 11, 12, 13, 14, 15]
-----------------------------
[2, 1, 2, 5, 5, 7, 4]
[16, 17, 18, 19, 20, 21, 22]
-----------------------------
[3, 4, 2, 8, 4]
[14, 23, 24, 25, 26]
-----------------------------
[9, 3, 2, 6, 3]
[27, 28, 29, 30, 31]
-----------------------------


In [66]:
def add_noise_to_states_childes(hidden_states, number_states, flip_prob=0.3):
    noisy_hidden_states = []
    for sequence in hidden_states:
        noisy_sequence = []
        for state in sequence:
            if np.random.rand() < flip_prob:
                possible_states = list(range(1, number_states + 1))  # Flip the state to a different random state
                possible_states.remove(state)                        # Remove the current state from possibilities
                new_state = np.random.choice(possible_states)
                noisy_sequence.append(new_state)
            else:
                noisy_sequence.append(state)
        noisy_hidden_states.append(noisy_sequence)
    return noisy_hidden_states

In [76]:
noisy_level = 0.8
noisy_hidden_states_universal = add_noise_to_states_childes(hidden_states_universal, len(pos_to_index), flip_prob=noisy_level)

In [77]:
for i in range(len(hidden_states_universal)):
    hidden_states_universal[i].insert(0, 0)
    noisy_hidden_states_universal[i].insert(0, 0)
    
    observations[i].insert(0, -1)

In [78]:
for index in range(len(observations[10030:10035])):
    print('[' + ', '.join(map(str, hidden_states_universal[10030:10035][index])) + ']')
    print('[' + ', '.join(map(str, noisy_hidden_states_universal[10030:10035][index])) + ']')
    print('-----------------------------')

[0, 4, 2, 6, 7, 3, 3, 4, 2, 2, 1, 5]
[0, 8, 9, 3, 1, 3, 6, 3, 7, 10, 5, 5]
-----------------------------
[0, 5, 1, 2, 8, 2, 1, 2, 2, 1, 2, 6, 3]
[0, 6, 6, 5, 6, 2, 4, 5, 9, 1, 6, 6, 4]
-----------------------------
[0, 7, 4, 7, 2, 2, 6, 6, 1, 6, 2, 5, 1, 6, 1]
[0, 9, 4, 6, 3, 2, 9, 2, 9, 9, 2, 5, 4, 9, 6]
-----------------------------
[0, 3, 6, 2, 6, 1, 2, 8, 6]
[0, 10, 6, 10, 3, 5, 2, 5, 6]
-----------------------------
[0, 1, 6, 2, 1, 5, 7, 4, 7, 5, 1, 2, 1, 8, 9, 6, 2, 1, 5, 5, 10, 4]
[0, 7, 6, 2, 8, 5, 6, 4, 7, 3, 8, 5, 8, 2, 4, 6, 4, 10, 5, 4, 2, 4]
-----------------------------


In [79]:
file_path = f"../data/Childes_synthetic_dataset(noise-{noisy_level}).npz"
obs_object = np.array(observations, dtype=object)
uni_hid_object = np.array(hidden_states_universal, dtype=object)
noisy_uni_hid_object = np.array(noisy_hidden_states_universal, dtype=object)
np.savez(file_path, num_states=len(pos_to_index) + 1, num_obs=len(word_to_index), observation=obs_object, real_hidden_universal=uni_hid_object, noisy_hidden_universal=noisy_uni_hid_object, noisy_level=noisy_level)